# Goal of ETL

The main goal of this notebook is to take the [World Bank Project data set](https://datacatalog.worldbank.org/dataset/world-bank-projects-operations) and merge this data with the [World Bank indicator data](https://data.worldbank.org/indicator/SP.POP.TOTL). Then load the merged data into a database.

In the process, we'll need to transform these data sets in different ways. And finally, we'll code an ETL pipeline to extract, transform, and load the data all in one step.

# Extracting data from a csv file

The first step in an ETL pipeline is extraction. Data comes in all types of different formats, and I'll note down stpes to extract data from csv files, JSON files, XML files, SQL databases, and the web.

In this first step, I'll practice extracting data from a CSV file and then navigating through the results. 

## Part 1 projects_data.csv

I'll be using the following csv files:
* projects_data.csv
* population_data.csv

As a first step, try importing the projects data using the pandas [read_csv method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html). The file path is just '../data/projects_data.csv'. 

In [1]:
# import the projects_data.csv file using the pandas library
# Store the results in the df_projects variable
import pandas as pd
df_projects = pd.read_csv('data/projects_data.csv')

C:\Users\ADMIN\anaconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (44) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


We get a DType warning. Read about what this warning is in the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.errors.DtypeWarning.html).

Pandas tries to figure out programatically the data type of each column (integer, float, boolean, string). In this case, pandas could not automatically figure out the data type. That is because some columns have more than one possible data types. In other words, this data is messy.

We can use the dtype option to specify the data type of each column. Because there are so many columns in this data set, we will set all columns to be strings at least for now.

Let's read in the data set again using the read_csv() method. This time, also use the option dtype=str so that pandas treats everything like a string.

In [2]:
# Read in the projects_data.csv file using the read_csv method and dtype = str option
df_projects = pd.read_csv('data/projects_data.csv', dtype=str)

In [3]:
# See what the data looks like
df_projects.head()

id  regionname                                        countryname  \
0  P162228       Other                                        World;World   
1  P163962      Africa  Democratic Republic of the Congo;Democratic Re...   
2  P167672  South Asia  People's Republic of Bangladesh;People's Repub...   
3  P158768  South Asia  Islamic  Republic of Afghanistan;Islamic  Repu...   
4  P161364      Africa  Federal Republic of Nigeria;Federal Republic o...   

  prodline                  lendinginstr lendinginstrtype  \
0       RE  Investment Project Financing               IN   
1       PE  Investment Project Financing               IN   
2       PE  Investment Project Financing               IN   
3       PE  Investment Project Financing               IN   
4       PE  Investment Project Financing               IN   

  envassesmentcategorycode supplementprojectflg productlinetype  \
0                        C                    N               L   
1                        B                    N               L   
2                      NaN                    Y               L   
3                        A                    N               L   
4                        B                    N               L   

  projectstatusdisplay  ... mjtheme3name mjtheme4name mjtheme5name  \
0               Active  ...          NaN          NaN          NaN   
1               Active  ...          NaN          NaN          NaN   
2               Active  ...          NaN          NaN          NaN   
3               Active  ...          NaN          NaN          NaN   
4               Active  ...          NaN          NaN          NaN   

                                            location  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  0002327546!$!Ogun State!$!7!$!3.58333!$!NG;000...   

                                            GeoLocID  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4  0002327546;0002328925;0002565340;0002565343;00...   

                                          GeoLocName                 Latitude  \
0                                                NaN                      NaN   
1                                                NaN                      NaN   
2                                                NaN                      NaN   
3                                                NaN                      NaN   
4  Ogun State;Niger State;Abia State;Edo;Kebbi St...  7;10;5.41667;6.5;11.5;8   

                Longitude            Country Unnamed: 56  
0                     NaN                NaN         NaN  
1                     NaN                NaN         NaN  
2                     NaN                NaN         NaN  
3                     NaN                NaN         NaN  
4  3.58333;6;7.5;6;4;10.5  NG;NG;NG;NG;NG;NG         NaN  

[5 rows x 57 columns]

In [5]:
# Count the number of null values in the data set
# HINT: use the isnull() and sum() methods
df_projects.isnull().sum().sort_values(ascending=False)

Unnamed: 56                 18248
mjtheme3name                18248
mjsector1                   18248
mjsector2                   18248
mjsector3                   18248
mjsector4                   18248
mjsector5                   18248
theme                       18248
mjtheme1name                18248
projectdoc                  18248
mjtheme2name                18248
mjtheme4name                18248
mjtheme5name                18248
majorsector_percent         18248
sector5                     15535
theme5                      15320
Country                     14045
GeoLocName                  13923
location                    13922
GeoLocID                    13922
Latitude                    13922
Longitude                   13922
sector4                     13872
theme4                      13014
sector3                     11761
financier                   11188
theme3                      10764
sector2                      8721
theme2                       8395
goal          

Notice that the number 18248 shows up multiple times. There is also a countryname column with 0 missing values and a Country column with 14045 missing values. This data set clearly has some issues that will need to be solved in the transform part of the pipeline.

Next, output the shape of the data frame.

In [7]:
# Output the shape of the data frame
df_projects.shape

(18248, 57)

There are 18248 rows in this data set. Considering many columns had 18248 NaN values, many columns in the data set are filled completely with NaN values.

## Part 2 population_data.csv

Next, use the pandas read_csv method to read in the population_data.csv file. The path to this file is "data/population_data.csv". 

In [8]:
# Read in the population_data.csv file using the read_csv() method
# Put the results in a variable called df_population
df_population = pd.read_csv('data/population_data.csv')

ParserError: Error tokenizing data. C error: Expected 3 fields in line 5, saw 63


There is something wrong with this data set. You should see an error that says "expected 3 fields in line 5, saw 63". What might have happened? Try printing out the first few lines of the data file to see what the issue might be.

In [13]:
# Print out the first 10 lines of the data set, line by line.
with open('data/population_data.csv') as f:
    for c, line in enumerate(f):
        if c<=10:
            print(f'line{c} -> {line}')

line0 -> ï»¿"Data Source","World Development Indicators",

line1 -> 

line2 -> "Last Updated Date","2018-06-28",

line3 -> 

line4 -> "Country Name","Country Code","Indicator Name","Indicator Code","1960","1961","1962","1963","1964","1965","1966","1967","1968","1969","1970","1971","1972","1973","1974","1975","1976","1977","1978","1979","1980","1981","1982","1983","1984","1985","1986","1987","1988","1989","1990","1991","1992","1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017",

line5 -> "Aruba","ABW","Population, total","SP.POP.TOTL","54211","55438","56225","56695","57032","57360","57715","58055","58386","58726","59063","59440","59840","60243","60528","60657","60586","60366","60103","59980","60096","60567","61345","62201","62836","63026","62644","61833","61079","61032","62149","64622","68235","72504","76700","80324","83200","85451","87277","89005","90853","92898","

The first four lines in the file are not properly formatted and don't contain data. Next, read in the data using the read_csv method. But this time, use the skiprows option.

In [14]:
# Read in population data skipping the first four rows
# Put the results in a variable called df_population
df_population = pd.read_csv('data/population_data.csv', skiprows=4, dtype=str)

In [15]:
# Run this cell to see what the data looks like
df_population.head()

Country Name Country Code     Indicator Name Indicator Code     1960  \
0        Aruba          ABW  Population, total    SP.POP.TOTL    54211   
1  Afghanistan          AFG  Population, total    SP.POP.TOTL  8996351   
2       Angola          AGO  Population, total    SP.POP.TOTL  5643182   
3      Albania          ALB  Population, total    SP.POP.TOTL  1608800   
4      Andorra          AND  Population, total    SP.POP.TOTL    13411   

      1961     1962     1963     1964     1965  ...      2009      2010  \
0    55438    56225    56695    57032    57360  ...    101453    101669   
1  9166764  9345868  9533954  9731361  9938414  ...  28004331  28803167   
2  5753024  5866061  5980417  6093321  6203299  ...  22549547  23369131   
3  1659800  1711319  1762621  1814135  1864791  ...   2927519   2913021   
4    14375    15370    16412    17469    18549  ...     84462     84449   

       2011      2012      2013      2014      2015      2016      2017  \
0    102053    102577    103187    103795    104341    104822    105264   
1  29708599  30696958  31731688  32758020  33736494  34656032  35530081   
2  24218565  25096150  25998340  26920466  27859305  28813463  29784193   
3   2905195   2900401   2895092   2889104   2880703   2876101   2873457   
4     83751     82431     80788     79223     78014     77281     76965   

  Unnamed: 62  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  

[5 rows x 63 columns]

Make sure to scroll over to see what the last column looks like. That last column, called 'Unnamed: 62', doesn't look very useful and is filled with NaN values.

In [17]:
# Count the number of null values in each column
df_population.isnull().sum().sort_values(ascending=False)

Unnamed: 62       264
1981                4
1975                4
1976                4
1977                4
                 ... 
2000                1
Indicator Name      0
Indicator Code      0
Country Code        0
Country Name        0
Length: 63, dtype: int64

It looks like every year column has at least one NaN value. 

In [18]:
# Count the number of null values in each row
# HINT: In the sum method, use axis=1
df_population.isnull().sum(axis=1)

0      1
1      1
2      1
3      1
4      1
      ..
259    1
260    1
261    1
262    1
263    1
Length: 264, dtype: int64

And it looks like almost every row has only one null value. That is probably from the 'Unnamed: 62' column that doesn't have any relevant information in it. Next, drop the 'Unnamed: 62' column from the data frame.

In [19]:
# Drop the 'Unnamed: 62' column from the data frame, 
# and save the results in the df_population variable

df_population = df_population.drop(columns=['Unnamed: 62'])

In [23]:
# Outputs any row that contains a null value
# The purpose is to see what rows contain null values now that 
#   'Unnamed: 62' was dropped from the data.
df_population[df_population.isnull().any(axis=1)]

Country Name Country Code     Indicator Name Indicator Code  \
67                     Eritrea          ERI  Population, total    SP.POP.TOTL   
108             Not classified          INX  Population, total    SP.POP.TOTL   
125                     Kuwait          KWT  Population, total    SP.POP.TOTL   
194         West Bank and Gaza          PSE  Population, total    SP.POP.TOTL   
212                     Serbia          SRB  Population, total    SP.POP.TOTL   
223  Sint Maarten (Dutch part)          SXM  Population, total    SP.POP.TOTL   

        1960     1961     1962     1963     1964     1965  ...     2008  \
67   1397491  1432640  1469645  1508273  1548187  1589179  ...  4232636   
108      NaN      NaN      NaN      NaN      NaN      NaN  ...      NaN   
125   269618   301336   338296   379891   425235   473554  ...  2652340   
194      NaN      NaN      NaN      NaN      NaN      NaN  ...  3596688   
212      NaN      NaN      NaN      NaN      NaN      NaN  ...  7350222   
223      NaN      NaN      NaN      NaN      NaN      NaN  ...    33964   

        2009     2010     2011     2012     2013     2014     2015     2016  \
67   4310334  4390840  4474690      NaN      NaN      NaN      NaN      NaN   
108      NaN      NaN      NaN      NaN      NaN      NaN      NaN      NaN   
125  2818939  2998083  3191051  3395556  3598385  3782450  3935794  4052584   
194  3702218  3811102  3927051  4046901  4169506  4294682  4422143  4551566   
212  7320807  7291436  7234099  7199077  7164132  7130576  7095383  7058322   
223    34238    34056    33435    34640    36607    37685    38824    39969   

        2017  
67       NaN  
108      NaN  
125  4136528  
194  4684777  
212  7022268  
223    41109  

[6 rows x 62 columns]

# Conclusion 

This population data doesn't look too bad. Only six rows have missing values. In the transformation part of the lesson, you'll have to deal with these missing values somehow.